In [20]:
# coding: UTF-8
import argparse
import logging
import os
import random
import sys
from datetime import datetime
from posixpath import join

import faiss
import numpy as np
import pandas as pd
# using labse
# from transformers import *
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data as Data
from torch.nn import *
sys.path.append('..')
from loader.DBP15KRawNeighbors import DBP15KRawNeighbors
from script.preprocess.deal_raw_dataset import MyRawdataset
from settings import *
from model.layers_LaBSE_neighbor import *

sys.argv=['']
del sys

### model

MyEmbedder: 继承nn.module

### update函数 
momentum = 0.9999
key*0.9999 + 0.0001*query

In [21]:
parser = argparse.ArgumentParser()
args = parse_options(parser)
model = MyEmbedder(args, VOCAB_SIZE)
modelS = MyEmbedder(args,VOCAB_SIZE)
list(model.parameters())

[Parameter containing:
 tensor([[[-6.9449e-04,  2.6912e-03, -4.6595e-04,  ..., -2.8645e-03,
            2.4195e-03, -6.6257e-04],
          [-1.0049e-03, -5.0237e-04, -1.4626e-03,  ..., -2.5157e-03,
            8.5154e-04, -9.8197e-04],
          [ 1.5551e-03, -1.6492e-03, -1.3634e-03,  ..., -2.5297e-03,
            6.2598e-04,  2.8866e-04],
          ...,
          [ 1.0801e-03, -1.4021e-03,  5.3374e-04,  ..., -4.2688e-04,
           -1.6067e-03, -2.4894e-03],
          [ 6.0395e-05, -5.3561e-04,  2.8277e-03,  ..., -3.7410e-04,
            3.0223e-03,  1.1275e-03],
          [-6.9074e-04, -1.6215e-03, -7.1493e-04,  ..., -2.4690e-03,
            1.7762e-04, -2.1898e-03]]], requires_grad=True),
 Parameter containing:
 tensor([[[ 0.0814],
          [-0.0721],
          [-0.0458],
          [ 0.0194],
          [ 0.0521],
          [ 0.0855],
          [ 0.0398],
          [ 0.0513],
          [-0.0810],
          [ 0.0101],
          [-0.0616],
          [ 0.0491],
          [-0.0733],
 

In [22]:
model.eval()

MyEmbedder(
  (attn): BatchMultiHeadGraphAttention(
    (leaky_relu): LeakyReLU(negative_slope=0.2)
    (softmax): Softmax(dim=-1)
    (dropout): Dropout(p=0.3, inplace=False)
  )
  (attn_mlp): Sequential(
    (0): Linear(in_features=1536, out_features=768, bias=True)
  )
  (criterion): NCESoftmaxLoss(
    (criterion): CrossEntropyLoss()
  )
)

In [23]:
model.state_dict()

OrderedDict([('attn.w',
              tensor([[[-6.9449e-04,  2.6912e-03, -4.6595e-04,  ..., -2.8645e-03,
                         2.4195e-03, -6.6257e-04],
                       [-1.0049e-03, -5.0237e-04, -1.4626e-03,  ..., -2.5157e-03,
                         8.5154e-04, -9.8197e-04],
                       [ 1.5551e-03, -1.6492e-03, -1.3634e-03,  ..., -2.5297e-03,
                         6.2598e-04,  2.8866e-04],
                       ...,
                       [ 1.0801e-03, -1.4021e-03,  5.3374e-04,  ..., -4.2688e-04,
                        -1.6067e-03, -2.4894e-03],
                       [ 6.0395e-05, -5.3561e-04,  2.8277e-03,  ..., -3.7410e-04,
                         3.0223e-03,  1.1275e-03],
                       [-6.9074e-04, -1.6215e-03, -7.1493e-04,  ..., -2.4690e-03,
                         1.7762e-04, -2.1898e-03]]])),
             ('attn.a_src',
              tensor([[[ 0.0814],
                       [-0.0721],
                       [-0.0458],
                

In [24]:
model.state_dict().keys()

odict_keys(['attn.w', 'attn.a_src', 'attn.a_dst', 'attn.bias', 'attn_mlp.0.weight', 'attn_mlp.0.bias'])

model.parameters()就是模型的两层，分别是attn的四个参数和mlp的两个参数

In [25]:
for key_param, query_param in zip(model.parameters(), modelS.parameters()):
    print(key_param.shape)
    
    key_param.data *= model.args.momentum
    key_param.data += (1 - model.args.momentum) * query_param.data

torch.Size([1, 768, 768])
torch.Size([1, 768, 1])
torch.Size([1, 768, 1])
torch.Size([768])
torch.Size([768, 1536])
torch.Size([768])
